# Hydro Planning

The General Eccentric Power and Lighting Company has a system
consisting of two dams and their associated reservoirs and power
plants on a river.  The important flows of power and water are shown
in the following diagram:

![SegmentLocal](hydro.gif "segment")

In the following table, all quantities measuring water are in units of 1000 acre-feet (KAF's).  Power is measured in megawatt hours (MWH's):<br>

|                       |A     |B     |Units    |
|-----------------------|------|------|---------|
|Storage Capacity       |2000  |1500  |KAF      |
|Minimum allowable level|1200  |800   |KAF      |
|Predicted inflow:      |      |      |         |
|        March          |200   |40    |KAF      |
|        April          |130   |15    |KAF      |
|March 1 level          |1900  |850   |KAF      |
|Water-Power Conversion |400   |200   |MWH/KAF  |
|Power Plant Capacity   |60,000|35,000|MWH/month|


Power can be sold at \\$5.00 per MWH for up to 50,000 MWH each month,
and excess power above that figure can be sold for \\$3.50 per MWH.
Assume flow rates in and out through the power plants are constant
within the month.  

### Suppose there is a spillway by each reservoir that allows water to spill out (at any level) and bypass the relevant power plant. A consequence of these assumptions is that the maximum and minimum water-level constraints need to be satisfied only at the end of the month. Formulate a linear program to maximize the amount of money General Eccentric receives for the power it sells during the months of March and April, given the constraints.

In [1]:
# Load the gams extension
%load_ext gams_magic


__The GAMS Model:__

In [2]:
%%gams

Set
    dams 'dams' / A, B /
    months 'months' / march, april /; 
   
Parameter storage_capacity(dams) 'Storage Capacity of water (units: KAF)'    / A 2000,
                                                                               B 1500 /;
                                                 
Parameter min_level(dams) 'Minimum allowable reservation level (units: KAF)'    / A 1200,
                                                                                  B  800 /;
                                                         
Table inflow(months,dams) 'Predicted inflow of water (units: KAF)'
            A    B
    march  200   40
    april  130   15;
    
Parameter resLev0(dams) 'March 1 reservation level of water (units: KAF)'    / A 1900,
                                                                               B  850 /;
                                                              
Parameter conversion(dams) 'Water-Power Conversion (units: MWH/KAF)'    / A 400,
                                                                          B 200 /;
                                                                          
Parameter power_capacity(dams) 'Power Plant Capacity (units: MWH/month)'    / A 60000,
                                                                              B 35000 /;
                                                                              
Parameter price1 'Power can be sold at $5.00 per MWH for up to 50,000 MWH each month' /5/;
Parameter price2 'excess power above that figure can be sold for $3.50 per MWH' /3.5/;

Variables
    resLev(dams,months) 'level in reservoir at end of month, KAF'
    genPower(dams,months) 'type of power generated each month, MWH'
    water(dams,months) 'amount of water sent to power plant each month, KAF'
    x(dams,months) 'amount of energy saled less than 50000, MWH'
    y(dams,months) 'amount of energy saled over 50000, MWH'
    z 'amount of money General Eccentric receives for the power it sells during both months'
    spill(dams,months) 'amount of water spilled at end of month, KAF'; 

Positive Variable water, genPower, resLev, x, y, spill;

Equation
    obj 'calculate the total money'
    eq_resLevA(months) 'flow of reservation level of water for A'
    eq_resLevB(months) 'flow of reservation level of water for B'
    eq_genPower(dams,months) 'calculate the energy generated with the conversion of water'
    eq_resLev_cap(dams,months) 'reservation level of water must be less than the capacity'
    eq_resLev_min(dams,months) 'reservation level of water must be more than the minimum'
    eq_power_cap(dams,months) 'reservation level of water must be more than the minimum'
    eq_x(months) 'the energy saled by 5 is extracted by both dams'
    eq_y(dams,months) 'for each dams, the energy generated is equal to the energy saled by 5, plus the saled by 3.5'
    ;

    obj.. sum((dams,months), x(dams,months)*price1 + y(dams,months)*price2 ) =e= z;
    eq_resLevA(months).. resLev0('A')$(ord(months)=1) + resLev('A',months-1) - water('A',months) + inflow(months,'A') - spill('A',months) =e= resLev('A',months);
    eq_resLevB(months).. resLev0('B')$(ord(months)=1) + resLev('B',months-1) - water('B',months) + inflow(months,'B') - spill('B',months) + spill('A',months) + water('A',months) =e= resLev('B',months);
    eq_genPower(dams,months).. genPower(dams,months) =e= water(dams,months)*conversion(dams);
    eq_resLev_cap(dams,months).. resLev(dams,months) =l= storage_capacity(dams);
    eq_resLev_min(dams,months).. resLev(dams,months) =g= min_level(dams);
    eq_power_cap(dams,months).. genPower(dams,months) =l= power_capacity(dams);
    eq_x(months).. sum(dams, x(dams,months)) =l= 50000;
    eq_y(dams,months).. genPower(dams,months) =e= x(dams,months) + y(dams,months); 

*eq_x2(dams,months) 'the energy saled by 5 must be less than the energy generated by each dams'
*eq_x2(dams,months).. x(dams,months) =l= genPower(dams,months);

model prob /all/;
Options LP  = Cplex;
Solve prob using LP maximizing z;
display resLev.l, spill.l, water.l, x.l, y.l, genPower.l, z.l;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),815000.0,27,25,LP,CPLEX,0.015


__Pull the variable values from GAMS:__

In [3]:
# Pull the values of x from GAMS to Python as a pandas dataframe
%gams_pull -d resLev genPower water spill
display(resLev,genPower,water, spill)

,dams,months,level,marginal,lower,upper,scale
0,A,march,1950.0,0.0,0.0,inf,1.0
1,A,april,1930.0,0.0,0.0,inf,1.0
2,B,march,810.0,0.0,0.0,inf,1.0
3,B,april,800.0,0.0,0.0,inf,1.0


,dams,months,level,marginal,lower,upper,scale
0,A,march,60000.0,0.0,0.0,inf,1.0
1,A,april,60000.0,0.0,0.0,inf,1.0
2,B,march,35000.0,0.0,0.0,inf,1.0
3,B,april,35000.0,0.0,0.0,inf,1.0


,dams,months,level,marginal,lower,upper,scale
0,A,march,150.0,0.0,0.0,inf,1.0
1,A,april,150.0,0.0,0.0,inf,1.0
2,B,march,175.0,0.0,0.0,inf,1.0
3,B,april,175.0,0.0,0.0,inf,1.0


,dams,months,level,marginal,lower,upper,scale
0,A,march,0.0,4.940656e-324,0.0,inf,1.0
1,A,april,0.0,4.940656e-324,0.0,inf,1.0
2,B,march,55.0,0.000000e+00,0.0,inf,1.0
3,B,april,0.0,4.940656e-324,0.0,inf,1.0
